In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from itertools import islice

from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from model import VQTrEstimator

In [6]:
from pytorch_lightning.utilities.model_summary import summarize
from gluonts.dataset.common import ListDataset


In [7]:
#Tuning GluonTS models with Optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import optuna
import torch
from gluonts.mx import Trainer


In [8]:
prediction_length  = 24

In [9]:
dataset = get_dataset("electricity")

In [16]:
class VQTrTuningObjective:  
    def __init__(self, dataset, prediction_length, metric_type="mean_wQuantileLoss"):
        self.dataset = dataset
        self.prediction_length = prediction_length
        
        self.metric_type = metric_type
    
    def get_params(self, trial) -> dict:
        return {   
        #  "context_length":trial.suggest_int("context_length", 240, 336,step = 48),   
        "num_encoder_layers": trial.suggest_int("num_encoder_layers", 2, 16, step = 4),
        "num_decoder_layers": trial.suggest_int("num_decoder_layers", 2, 16,step = 4),
        # "dim_feedforward" :  trial.suggest_int("dim_feedforward", 2, 32,step = 16), 
        # "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        }
     
    def __call__(self, trial):
        params = self.get_params(trial)
        estimator = VQTrEstimator(
          freq=self.dataset.metadata.freq,
          prediction_length=prediction_length,
          context_length=prediction_length*12,
          codebook_size=25,
          dim_head=8,
          nhead=2,
          depth=1,
          num_encoder_layers=params['num_encoder_layers'],
          num_decoder_layers=params['num_decoder_layers'],
          dim_feedforward=16,
          activation="gelu",

          num_feat_static_cat=1,
          cardinality=[321],
          embedding_dimension=[3],
          
          scaling=True,
        
          batch_size=256,
          num_batches_per_epoch=300,
          trainer_kwargs=dict(max_epochs=1, accelerator='gpu', devices=1),
      )
        predictor = estimator.train(
            training_data=self.dataset.train,
            num_workers=8,
            shuffle_buffer_length=512,
            cache_data=True,
        )
        
        forecast_it, ts_it = make_evaluation_predictions(
            dataset=self.dataset.test,
            predictor=predictor
        )
        forecasts = list(forecast_it)
        # if layer == layers[0]:
        tss = list(ts_it)
        
        evaluator = Evaluator()
        agg_metrics, _ = evaluator(iter(tss), iter(forecasts))
        return agg_metrics[self.metric_type]

In [17]:
import time
start_time = time.time()
study = optuna.create_study(direction="minimize")
study.optimize(VQTrTuningObjective(dataset, prediction_length = dataset.metadata.prediction_length), n_trials=10)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
print(time.time() - start_time)

[I 2022-08-21 20:49:31,814] A new study created in memory with name: no-name-ccbe6759-cce4-4f94-953e-c2b6fcd18ae1
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:563: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  low=low, old_high=old_high, high=high, step=step
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to c

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 300: 'train_loss' reached 9.48965 (best 9.48965), saving model to '/content/drive/MyDrive/vq-tr-main/lightning_logs/version_7/checkpoints/epoch=0-step=300.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
Running evaluation: 2247it [00:00, 25057.43it/s]
/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:1201: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
[I 2022-08-21 21:00:09,566] Trial 0 finished with value: 0.25837099847978373 and parameters: {'num_encoder_layers': 6, 'num_decoder_layers': 14}. Best is trial 0 with value: 0.25837099847978373.
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:563: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  low=low, old_high=old_high, high=high, step=step
/usr/

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 300: 'train_loss' reached 9.97407 (best 9.97407), saving model to '/content/drive/MyDrive/vq-tr-main/lightning_logs/version_8/checkpoints/epoch=0-step=300.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
Running evaluation: 2247it [00:00, 25275.02it/s]
/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:1201: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
[I 2022-08-21 21:08:26,952] Trial 1 finished with value: 0.27443531041628977 and parameters: {'num_encoder_layers': 14, 'num_decoder_layers': 10}. Best is trial 0 with value: 0.25837099847978373.
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:563: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  low=low, old_high=old_high, high=high, step=step
/usr

...

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 300: 'train_loss' reached 12.73336 (best 12.73336), saving model to '/content/drive/MyDrive/vq-tr-main/lightning_logs/version_15/checkpoints/epoch=0-step=300.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
Running evaluation: 2247it [00:00, 26017.34it/s]
/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:1201: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
[I 2022-08-21 21:51:41,484] Trial 8 finished with value: 0.09802857866215788 and parameters: {'num_encoder_layers': 10, 'num_decoder_layers': 6}. Best is trial 3 with value: 0.08734118106732694.
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:563: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  low=low, old_high=old_high, high=high, step=step
/u

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 300: 'train_loss' reached 9.94036 (best 9.94036), saving model to '/content/drive/MyDrive/vq-tr-main/lightning_logs/version_16/checkpoints/epoch=0-step=300.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
Running evaluation: 2247it [00:00, 25550.55it/s]
/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:1201: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
[I 2022-08-21 22:00:33,560] Trial 9 finished with value: 0.11801514944531141 and parameters: {'num_encoder_layers': 14, 'num_decoder_layers': 10}. Best is trial 3 with value: 0.08734118106732694.


Number of finished trials: 10
Best trial:
  Value: 0.08734118106732694
  Params: 
    num_encoder_layers: 6
    num_decoder_layers: 10
4261.750772714615
